# Generating the AUC tables



In [3]:
%load_ext autoreload

%autoreload 2
from utils import parse_table, read_csv
import pandas as pd
import numpy as np

from pretty import plot_kde, ColorTheme

df = read_csv('score-norm_vs_likelihood.csv')
all_tasks = parse_table(df)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1/6:   0%|          | 0/4 [00:00<?, ?it/s]

5/6: 100%|██████████| 4/4 [00:08<00:00,  2.07s/it]


In [8]:

def get_scatter(in_distr, ood, all_tasks, metric='score-norm'): 
    global likelihood_in, metric_in
    global likelihood_ood, metric_ood
    in_vs_out = all_tasks[in_distr][ood]
    likelihood_in = in_vs_out[in_vs_out['name'] == 'test']['log-likelihood'].values
    metric_in = in_vs_out[in_vs_out['name'] == 'test'][metric].values
    likelihood_ood = in_vs_out[in_vs_out['name'] == 'ood']['log-likelihood'].values
    metric_ood = in_vs_out[in_vs_out['name'] == 'ood'][metric].values


In [5]:
from roc_analysis import get_roc_graph, get_convex_hull, get_auc
from tqdm import tqdm

pairs = [
    ('mnist', 'fashion-mnist'),
    ('mnist', 'omniglot'),
    ('fashion-mnist', 'mnist'),
    ('fashion-mnist', 'omniglot'),
    ('cifar10', 'svhn'),
    ('cifar10', 'celeba-small'),
    ('svhn', 'cifar10'),
    ('svhn', 'celeba-small'),
]

for in_distr, ood in pairs:
    print("----")
    print(in_distr, ood)
    np.random.seed(111)
    get_scatter(in_distr, ood, all_tasks)
    x_naive, y_naive = get_roc_graph(
        pos_x = metric_in,
        neg_x = metric_ood,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_naive, y_naive)
    naive_auc = get_auc(x_curve, y_curve)
    print("Naive AUC", naive_auc)
    
    x_ours, y_ours = get_roc_graph(
        pos_x = likelihood_in,
        pos_y = metric_in,
        neg_x = likelihood_ood,
        neg_y = metric_ood,
        compute_limit=10000,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_ours, y_ours)
    our_auc = max(naive_auc, get_auc(x_curve, y_curve))
    print("Dual threshold AUC", our_auc)

----
mnist fashion-mnist
Naive AUC 0.9192435443401337
Dual threshold AUC 0.996117502450943
----
mnist omniglot
Naive AUC 0.004457212582663374
Dual threshold AUC 0.9989175123792466
----
fashion-mnist mnist
Naive AUC 0.07481348514556885
Dual threshold AUC 0.23966631293296814
----
fashion-mnist omniglot
Naive AUC 0.0005937631694113064
Dual threshold AUC 0.9489644157441326
----
cifar10 svhn
Naive AUC 0.8826910257339478
Dual threshold AUC 0.8826910257339478
----
cifar10 celeba-small
Naive AUC 0.7158326221113441
Dual threshold AUC 0.7158326221113441
----
svhn cifar10
Naive AUC 0.11981871724128723
Dual threshold AUC 0.9942491352558136
----
svhn celeba-small
Naive AUC 0.14480872915572474
Dual threshold AUC 0.9883386948529411


In [6]:

df = read_csv('grayscale_LID_vs_likelihood_flow_lpca_medium_scale.csv')
all_tasks = parse_table(df)
df = read_csv('rgb_LID_vs_likelihood_flow_lpca_medium_scale.csv')
all_tasks_ = parse_table(df)
for key in all_tasks_:
    all_tasks[key] = all_tasks_[key]


5/7: 100%|██████████| 5/5 [00:17<00:00,  3.53s/it]


In [10]:

df = read_csv('grayscale_LID_vs_likelihood_diffusion_lpca.csv')
all_tasks = parse_table(df)
df = read_csv('rgb_LID_vs_likelihood_diffusion_lpca.csv')
all_tasks_ = parse_table(df)
for key in all_tasks_:
    all_tasks[key] = all_tasks_[key]


5/7: 100%|██████████| 5/5 [00:34<00:00,  6.91s/it]


In [11]:
from roc_analysis import get_roc_graph, get_convex_hull, get_auc
from tqdm import tqdm

pairs = [
    ('mnist', 'fashion-mnist'),
    ('mnist', 'omniglot'),
    ('fashion-mnist', 'mnist'),
    ('fashion-mnist', 'omniglot'),
    ('cifar10', 'svhn'),
    ('cifar10', 'celeba-small'),
    ('svhn', 'cifar10'),
    ('svhn', 'celeba-small'),
]

for in_distr, ood in pairs:
    print("----")
    print(in_distr, ood)
    np.random.seed(111)
    get_scatter(in_distr, ood, all_tasks, metric='LID')
    x_naive, y_naive = get_roc_graph(
        pos_x = metric_in,
        neg_x = metric_ood,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_naive, y_naive)
    naive_auc = get_auc(x_curve, y_curve)
    print("Naive AUC", naive_auc)
    
    x_ours, y_ours = get_roc_graph(
        pos_x = likelihood_in,
        pos_y = metric_in,
        neg_x = likelihood_ood,
        neg_y = metric_ood,
        compute_limit=10000,
        verbose=0,
    )
    x_curve, y_curve = get_convex_hull(x_ours, y_ours)
    our_auc = max(naive_auc, get_auc(x_curve, y_curve))
    print("Dual threshold AUC", our_auc)

----
mnist fashion-mnist
Naive AUC 0.004191070795059204
Dual threshold AUC 0.9959636330604553
----
mnist omniglot
Naive AUC 0.0004468735817557038
Dual threshold AUC 0.9993341849552645
----
fashion-mnist mnist
Naive AUC 0.8523643910884857
Dual threshold AUC 0.8523643910884857
----
fashion-mnist omniglot
Naive AUC 0.1655849599244684
Dual threshold AUC 0.9581770520981907
----
cifar10 svhn
Naive AUC 0.9461166050000005
Dual threshold AUC 0.9461166050000005
----
cifar10 celeba-small
Naive AUC 0.6528089285714285
Dual threshold AUC 0.6528089285714285
----
svhn cifar10
Naive AUC 0.005385062499999995
Dual threshold AUC 0.9955338300000001
----
svhn celeba-small
Naive AUC 0.004131512605042014
Dual threshold AUC 0.995751155462185
